<a href="https://colab.research.google.com/github/PopoolaIbrahim/E-Mail_List_Generator/blob/main/Email_Scraper_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### Install Requirements
#!pip install google
!pip install fake_useragent
#!apt-get install telnet
#!apt-get install dnsutils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.2 MB/s eta 0:00:00


In [5]:

import csv
import re
import requests
from fake_useragent import UserAgent
import time

business = "Plumbers"
usa_states = [
    'Alabama',
    'Alaska',
    'Arizona',
    'Arkansas',
    'California',
    'Colorado',
    'Connecticut',
    'Delaware',
    'Florida',
    'Georgia',
    'Hawaii',
    'Idaho',
    'Illinois',
    'Indiana',
    'Iowa',
    'Kansas',
    'Kentucky',
    'Louisiana',
    'Maine',
    'Maryland',
    'Massachusetts',
    'Michigan',
    'Minnesota',
    'Mississippi',
    'Missouri',
    'Montana',
    'Nebraska',
    'Nevada',
    'New Hampshire',
    'New Jersey',
    'New Mexico',
    'New York',
    'North Carolina',
    'North Dakota',
    'Ohio',
    'Oklahoma',
    'Oregon',
    'Pennsylvania',
    'Rhode Island',
    'South Carolina',
    'South Dakota',
    'Tennessee',
    'Texas',
    'Utah',
    'Vermont',
    'Virginia',
    'Washington',
    'West Virginia',
    'Wisconsin',
    'Wyoming'
]

# Extracting E-Mail
def extract_emails(text):
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    return re.findall(email_regex, text)
email_list = []
for state in usa_states:
    url= f"https://www.google.com/search?q={business}+in+{state}+US+%22@gmail.com%22&num=100&gl=us"
    # Create a UserAgent object
    user_agent = UserAgent()
    # Generate a fake user agent string
    fake_ua = user_agent.random
    # Set headers with the fake user agent
    headers = {'User-Agent': fake_ua}
    response = requests.get(url, headers=headers)
    url_email = extract_emails(response.text)
    email_list.append(url_email)
    time.sleep(300)

#print(email_list)
list_of_lists = email_list

# Using nested list comprehension
single_list = [item for sublist in list_of_lists for item in sublist]
#print(len(single_list))
#print(single_list)

clean_list = []
for item in single_list:
    if "%" in item or item in clean_list or "com." in item or ":" in item or "=" in item or "*" in item :
        pass
    else :
        clean_list.append(item)

# Converting list to CSV
def append_to_csv(file_name, variable):
    with open(file_name, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([variable])
for item in clean_list:
  csv_file = '/content/drive/MyDrive/Email_List_Plumbers_in_USA.csv'
  append_to_csv(csv_file, item)

print(len(clean_list), " Emails")

182


In [ ]:
#### Email Cleaning
import csv
import pandas as pd
# Load the Excel file
df = pd.read_excel('Marketing Manager List.xlsx')
# Extract the first column into a list
single_list = df.iloc[:, 0].tolist()
clean_list_period = []
added_emails = set()
for item in single_list:
    if "@gmail" in item or "@yahoo" in item:
        email = item.split("@")[0] + item.split("@")[1].split(".")[0]  # Extract the username part of the email
        if email not in added_emails:
            clean_list_period.append(item)
            added_emails.add(email)
# Remove peroid from list
def remove_periods(strings):
    updated_strings = []
    for string in strings:
        updated_string = string.replace('.', '')
        updated_strings.append(updated_string)
    return updated_strings
clean_list = remove_periods(clean_list_period)
# Saving to CSV
def append_to_csv(file_name, variable):
    with open(file_name, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([variable])
for item in clean_list:
  csv_file = 'email_dataset.csv'
  append_to_csv(csv_file, item)

In [ ]:
### Email Verification
#!dig gmail.com MX
import subprocess

def check_gmail_existence(email):
    try:
        # Establish a telnet connection to Gmail SMTP server
        telnet = subprocess.Popen(["telnet", "gmail-smtp-in.l.google.com", "25"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)

        # Read the initial response from the server
        output = telnet.stdout.read().decode()

        if "220" not in output:
            return False

        # Send a VRFY command with the email address
        telnet.stdin.write(f"VRFY {email}\n".encode())
        telnet.stdin.flush()

        # Read the response from the server
        response = telnet.stdout.read().decode()

        # Check if the response indicates that the email address exists
        if "250" in response:
            return True
        else:
            return False
    except Exception as e:
        print("Error:", str(e))
        return False

# Example usage
email_address = "popoolaibrahim.t@gmail.com"
exists = check_gmail_existence(email_address)
if exists:
    print(f"The Gmail address '{email_address}' exists.")
else:
    print(f"The Gmail address '{email_address}' does not exist.")


Error: [Errno 32] Broken pipe
The Gmail address 'popoolaibrahim.t@gmail.com' does not exist.


In [ ]:
### Email Broadcast

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(sender_email, sender_password, receiver_email, subject, message):
    # Create a multipart message container
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach the message to the container
    msg.attach(MIMEText(message, 'plain'))

    try:
        # Create an SMTP session
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)

        # Send the email
        server.sendmail(sender_email, receiver_email, msg.as_string())
        print("Email sent successfully!")
    except smtplib.SMTPException as e:
        print("Error sending email:", str(e))
    finally:
        # Close the SMTP session
        server.quit()

# Example usage
sender_email = "your_email@gmail.com"
sender_password = "your_password"
receiver_email = "recipient_email@example.com"
subject = "Test Email"
message = "This is a test email sent from Python."

send_email(sender_email, sender_password, receiver_email, subject, message)

